In [1]:
import cv2
import numpy as np


In [2]:
# Load the image
image = cv2.imread(r'D:\Adobe-gen\Screenshot 2024-07-31 224237.png', cv2.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# Detect edges using Canny edge detection
edges = cv2.Canny(blurred, 50, 150)


In [3]:
# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty image to draw the contours
contour_image = np.zeros_like(image)

# Draw the detected contours
cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 1)



array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0, 255, ...,   0,   0,   0],
       [  0,   0, 255, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [5]:
# Fit and complete each contour
completed_image = np.zeros_like(image)

for contour in contours:
    # Get the x and y coordinates of the contour points
    contour = contour[:, 0, :]
    x = contour[:, 0]
    y = contour[:, 1]
    
    # Sort the points by x-coordinate
    sorted_indices = np.argsort(x)
    x_sorted = x[sorted_indices]
    y_sorted = y[sorted_indices]

    # Fit a polynomial curve (e.g., degree 2)
    poly_coefficients = np.polyfit(x_sorted, y_sorted, deg=2)
    polynomial = np.poly1d(poly_coefficients)

    # Generate x values to predict y values
    x_new = np.linspace(x_sorted.min(), x_sorted.max(), num=100)
    y_new = polynomial(x_new).astype(int)

    # Draw the fitted curve on the image
    for i in range(len(x_new) - 1):
        pt1 = (int(x_new[i]), int(y_new[i]))
        pt2 = (int(x_new[i + 1]), int(y_new[i + 1]))
        cv2.line(completed_image, pt1, pt2, 255, 1)

cv2.imshow("Completed Curves", completed_image)





In [6]:
# Combine the original and completed curves
combined_image = cv2.addWeighted(contour_image, 0.5, completed_image, 0.5, 0)

cv2.imshow("Combined Curves", combined_image)



In [7]:
import cv2
import numpy as np
from scipy.interpolate import splprep, splev



In [8]:
image = cv2.imread(r'D:\Adobe-gen\Screenshot 2024-07-31 224237.png', cv2.IMREAD_GRAYSCALE)

In [9]:


# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# Detect edges using Canny edge detection
edges = cv2.Canny(blurred, 50, 150)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty image to draw the contours
contour_image = np.zeros_like(image)

# Draw the detected contours
cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 1)

# Create an empty image to draw the completed curves
completed_image = np.zeros_like(image)

for contour in contours:
    if len(contour) < 5:
        # Skip contours that are too small to fit a spline
        continue

    # Get the x and y coordinates of the contour points
    contour = contour[:, 0, :]
    x = contour[:, 0]
    y = contour[:, 1]

    try:
        # Perform spline interpolation
        tck, u = splprep([x, y], s=0)
        u_new = np.linspace(u.min(), u.max(), 100)
        x_new, y_new = splev(u_new, tck)

        # Draw the fitted spline on the image
        for i in range(len(x_new) - 1):
            pt1 = (int(x_new[i]), int(y_new[i]))
            pt2 = (int(x_new[i + 1]), int(y_new[i + 1]))
            cv2.line(completed_image, pt1, pt2, 255, 1)
    except Exception as e:
        print(f"Error in spline fitting: {e}")
        continue

# Combine the original and completed curves
combined_image = cv2.addWeighted(contour_image, 0.5, completed_image, 0.5, 0)

cv2.imshow("Completed Curves", combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
